In [1]:
print("ok")

ok


In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
#from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from pypdf import *
import os

c:\Users\danny\.conda\envs\NLP\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
#PINECONE_API_KEY = "11dc1caa-0f08-413d-ab55-9f585ddcec7c"

In [6]:
os.environ["PINECONE_API_KEY"] = "11dc1caa-0f08-413d-ab55-9f585ddcec7c"

In [7]:
def load_pdf(data):
    loader = DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    
    documents = loader.load()
    
    return documents

In [8]:
extract_data= load_pdf("data/")

In [9]:
#extract_data
from langchain_pinecone import PineconeVectorStore
from langchain_pinecone import Pinecone

In [10]:
# Creating chunks of texts from the documents

def text_split(extract_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size =500,chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extract_data)
    
    return text_chunks


In [11]:
text_chunks = text_split(extract_data)
print("length of my chunk: ", len(text_chunks))

length of my chunk:  7020


In [12]:
# Downloading embeding model
def download_hugging_face_model():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings
 

In [13]:
embeddings  =  download_hugging_face_model()

c:\Users\danny\.conda\envs\NLP\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\danny\.conda\envs\NLP\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
print(embeddings)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


In [15]:
query_result = embeddings.embed_query("Hello World")
print("lenght", len(query_result))

lenght 384


In [ ]:
#query_result

In [16]:
from pinecone import ServerlessSpec

pc = pinecone.Pinecone(api_key=os.environ["PINECONE_API_KEY"])


In [18]:

index_name = "medchatbot"
index = pc.Index(index_name)

In [ ]:

#docsearch = Pinecone.from_texts([t.page_content for t in text_chunks],embeddings,index_name=index_name)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name="medchatbot",
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 


docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks],embeddings,index_name=index_name)


In [19]:
docsearch = Pinecone.from_existing_index(index_name,embeddings)
query = "what are allergies"
docs = docsearch.similarity_search(query,k=3)
print("Results", docs)

Results [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly c

In [20]:
prompt_template = """
Use the following piece of information to answer your question.
If you don't know the answer, just say I don't know, don't try to answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else

Helpful answer:
"""

In [21]:
PROMPT = PromptTemplate(template=prompt_template,input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

In [22]:
llm=CTransformers(
    model = "model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type = "llama",
    config = {
        "max_new_tokens":512,
        "temperature": 0.8
    }
)

In [1]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 1}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

NameError: name 'RetrievalQA' is not defined

In [27]:
while True:
    user_input = input(f'Input Prompt:')
    result = qa({'query': user_input})
    print('Response : ', result['result'])

Response :  I don't know. Acidosis is not related to acne. Respiratory acidosis, renal tubular acidosis, and metabolic acidosis are all unrelated medical conditions.
Response :  The correct answer is I don't know.


KeyboardInterrupt: 